In [1]:
import pandas as pd
from datetime import timedelta

In [6]:
import os
os.getcwd()

'C:\\Users\\XFE\\Documents\\mimic4ed-benchmark\\Benchmark_scripts'

In [14]:
ed_path = 'C:/Users/XFE/Documents/mimic4ed-benchmark/data/ed/'

ed_filename_dict = {'diagnosis':'diagnosis.csv', 'edstays':'edstays.csv',  'medrecon':'medrecon.csv',  'pyxis':'pyxis.csv',  'triage':'triage.csv',  'vitalsign':'vitalsign.csv'}

mimic_iv_path = 'C:/Users/XFE/Documents/mimic4ed-benchmark/data/'
mimic_iv_core = mimic_iv_path + 'core/'
mimic_iv_hosp = mimic_iv_path + 'hosp/'
mimic_iv_icu = mimic_iv_path + 'icu/'

icu_filename_dict = {"chartevents":"chartevents.csv","datetimeevents":"datetimeevents.csv","d_items":"d_items.csv","icustays":"icustays.csv","inputevents":"inputevents.csv","outputevents":"outputevents.csv","procedureevents":"procedureevents.csv"}
core_filename_dict = {"patients":"patients.csv", "admissions":"admissions.csv", "transfers":"transfers.csv"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv","d_icd_diagnoses":"d_icd_diagnoses.csv","d_labitems":"d_labitems.csv","emar":"emar.csv","hcpcsevents":"hcpcsevents.csv","microbiologyevents":"microbiologyevents.csv","poe":"poe.csv","prescriptions":"prescriptions.csv","services":"services.csv","diagnoses_icd":"diagnoses_icd.csv","d_icd_procedures":"d_icd_procedures.csv","drgcodes":"drgcodes.csv","emar_detail":"emar_detail.csv","labevents":"labevents.csv","pharmacy":"pharmacy.csv","poe_detail":"poe_detail.csv","procedures_icd":"procedures_icd.csv"}

## 1. Cohort Setup
Exclude:
1. pediatric patients, Age <18
2. Death before ED disposition

### (1) Merge age data 

In [15]:
# Merge edstays with patients age
df_edstays = pd.read_csv(ed_path+ed_filename_dict['edstays'])
df_patients = pd.read_csv(mimic_iv_core+core_filename_dict["patients"])

# Derive age from intime, anchor_year and anchor_age
df = pd.merge(df_edstays, df_patients[['subject_id', 'anchor_age', 'gender', 'anchor_year']], on = ['subject_id'], how='left')
df['in_year'] = pd.to_datetime(df['intime']).dt.year
df['age'] = df['in_year']-df['anchor_year']+df['anchor_age']
df.drop(['anchor_age', 'anchor_year', 'in_year'],axis=1, inplace=True)
##XF: may not need to drop in the master dataset?
len(df_edstays), len(df_patients), len(df)

(448972, 382278, 448972)

### (2) Merge deathtime data

In [16]:
# Merge edstays with patients deathtime

df_admissions = pd.read_csv(mimic_iv_core+core_filename_dict["admissions"])
df_death = df_admissions[pd.notnull(df_admissions['deathtime'])][['subject_id', 'deathtime']]

# Some subject have multiple deathtime, take latest.
## XF:seems not very reasonable. We probably should link to patient.csv and look at dod?
value_counts = df_death.value_counts(subset=['subject_id'])
print("Value counts:")
print(value_counts[0:6])
df_death['deathtime'] = pd.to_datetime(df_death['deathtime'])
gp = df_death.groupby(['subject_id'])
df_death_max = gp.max()

# Merge edstays with latest patient deathtime
df = pd.merge(df, df_death_max, on = ['subject_id'], how='left')

len(df_death), len(df_death_max), len(df)


Value counts:
subject_id
16714180      2
16940596      2
14566049      2
15394378      2
13771243      2
12453379      2
dtype: int64


(9337, 9331, 448972)

### (3) Exclude pediatric and death before disposition

In [17]:
## Move to next script 
df_adults = df[df['age']>=18].copy()
df_adults.loc[:,'outtime'] = pd.to_datetime(df_adults['outtime'])
df_live = df_adults[pd.isnull(df_adults['deathtime']) | (df_adults['outtime'] < df_adults['deathtime'])]

print(len(df), len(df_adults), len(df_live))
df_live.to_csv('df_live.csv', index=False)

448972 448804 448779


## 2. Triage Features
Extract features:
1. demographic (gendar and age have been included)
2. triage vital sign
3. health utilization (# of ed visits, hospitalization, ICU admission in past 3 month/1 year)
4. commorbidities (TBD)


In [18]:
#set df to previous obtained df_live
df = df_live.copy()

### (1) Generate number of ED visits in past

In [19]:
# Sort the ED visits according to subject id and intime
df.loc[:,'intime'] = pd.to_datetime(df['intime'])
df_visits = df.sort_values(['subject_id', 'intime']).reset_index()
df_visits.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,age,deathtime
0,335314,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,52,NaT
1,335315,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,52,NaT
2,335316,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,52,NaT
3,335317,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,52,NaT
4,335318,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,52,NaT


In [21]:
## XF: also generate past 30-day/90-day ED visit count
timerange = timedelta(days=365)
n_ed = [0 for _ in range(len(df_visits))]
# Loop through the sorted ED visits
for i, row in df_visits.iterrows():
    if i % 10000 == 0:
        print('Process: %d/%d' % (i, len(df_visits)), end='\r')
    # count number of subject's ED visits before the current iteration
    j=i-1
    while j>=0 and df_visits['subject_id'][j] == row['subject_id'] and row['intime']-df_visits['intime'][j]<=timerange:
        n_ed[i]+=1
        j-=1
df_visits.loc[:,'n_ed'] = n_ed


### (2) Generate number of admissions in the past

In [9]:
# Sort the admission according to subject id and admit time

df_admissions = pd.read_csv(mimic_iv_core+core_filename_dict["admissions"])
sorted_df = df_admissions[df_admissions['subject_id'].isin(df['subject_id'].unique().tolist())][['subject_id', 'admittime']].copy()


sorted_df.loc[:,'admittime'] = pd.to_datetime(sorted_df['admittime'])
sorted_df.sort_values(['subject_id', 'admittime'], inplace=True)
sorted_df.reset_index(drop=True, inplace=True)
len(sorted_df)


350510

In [10]:
timerange = timedelta(days=365)

## packcage this function to *.py ： give some parameter, how to use the function 

j_start = 0
j_end = 0
prev_subject=None
n_adm = [0 for _ in range(len(df_visits))]
# Loop through ED visits
for i, row in df_visits.iterrows():
    if i % 10000 == 0:
        print('Process: %d/%d' % (i, len(sorted_df)), end='\r')
    # If new subject, find the start and end index of same subject in sorted admission df
    if row['subject_id'] != prev_subject:
        j_start=j_end
        while j_start<len(sorted_df) and sorted_df['subject_id'][j_start] < row['subject_id']:
            j_start+=1
        j_end = j_start
        while j_end<len(sorted_df) and sorted_df['subject_id'][j_end] == row['subject_id']:
            j_end+=1
        prev_subject=row['subject_id']
    # Count number of previous admissions within the time range
    for j in range(j_start, j_end):
        if row['intime']>sorted_df['admittime'][j] and row['intime']-sorted_df['admittime'][j]<=timerange:
            n_adm[i]+=1
df_visits.loc[:,'n_adm'] = n_adm
    

### (3) Generate number of ICU in the past

In [11]:
# Sort the ICU according to subject id and intime

df_icu = pd.read_csv(mimic_iv_icu+icu_filename_dict["icustays"])
sorted_df = df_icu[df_icu['subject_id'].isin(df['subject_id'].unique().tolist())][['subject_id', 'intime']].copy()


sorted_df.loc[:,'intime'] = pd.to_datetime(sorted_df['intime'])
sorted_df.sort_values(['subject_id', 'intime'], inplace=True)
sorted_df.reset_index(drop=True, inplace=True)
len(sorted_df)


51604

In [12]:
j_start = 0
j_end = 0
prev_subject=None
n_icu = [0 for _ in range(len(df_visits))]
# Loop through ED visits
for i, row in df_visits.iterrows():
    if i % 10000 == 0:
        print('Process: %d/%d' % (i, len(sorted_df)), end='\r')
    # If new subject, find the start and end index of same subject in sorted admission df
    if row['subject_id'] != prev_subject:
        j_start=j_end
        while j_start<len(sorted_df) and sorted_df['subject_id'][j_start] < row['subject_id']:
            j_start+=1
        j_end = j_start
        while j_end<len(sorted_df) and sorted_df['subject_id'][j_end] == row['subject_id']:
            j_end+=1
        prev_subject=row['subject_id']
    # Count number of previous admissions within the time range
    for j in range(j_start, j_end):
        if row['intime']>sorted_df['intime'][j] and row['intime']-sorted_df['intime'][j]<=timerange:
            n_icu[i]+=1
df_visits.loc[:,'n_icu'] = n_icu
    

In [13]:
df_visits[:30]


,index,subject_id,hadm_id,stay_id,intime,outtime,gender,age,deathtime,n_ed,n_adm,n_icu
0,335314,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,52,NaT,0,0,0
1,335315,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,52,NaT,1,1,0
2,335316,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,52,NaT,2,2,0
3,335317,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,52,NaT,3,2,0
4,335318,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,52,NaT,4,3,1
5,56671,10000084,23052089.0,35203156,2160-11-20 20:36:00,2160-11-21 03:20:00,M,72,NaT,0,0,0
6,56672,10000084,29888819.0,36954971,2160-12-27 18:32:00,2160-12-28 16:07:00,M,72,NaT,1,1,0
7,167410,10000108,NaN,32522732,2163-09-16 16:34:00,2163-09-16 16:43:00,M,25,NaT,0,0,0
8,167411,10000108,NaN,39513268,2163-09-24 16:14:00,2163-09-24 16:19:00,M,25,NaT,1,0,0
9,167412,10000108,27250926.0,36533795,2163-09-27 16:18:00,2163-09-28 09:04:00,M,25,NaT,2,0,0


### (4) Merge triage vital sign

In [14]:
# Merge triage vital sign
df_triage = pd.read_csv(ed_path+ed_filename_dict['triage'])
df_visits = pd.merge(df_visits, df_triage, on = ['subject_id', 'stay_id'], how='left')

## Generate Labels

In [15]:
df_visits.loc[:,'admitted'] = df_visits['hadm_id'].notnull()
#df_icu = pd.read_csv(mimic_iv_icu+icu_filename_dict["icustays"])
df_visits.loc[:,'icu'] = df['hadm_id'].isin(df_icu['hadm_id'])
range_2d = timedelta(days=2)
range_7d = timedelta(days=7)
df_visits.loc[:,'outtime'] = pd.to_datetime(df_visits['outtime'])
df_visits.loc[:,'deathtime'] = pd.to_datetime(df_visits['deathtime'])

df_visits.loc[:,'death_2day'] = df_visits['deathtime'].notna and ((df_visits['deathtime'] - df_visits['outtime'])<=range_2d)
df_visits.loc[:,'death_7day'] = df_visits['deathtime'].notna and ((df_visits['deathtime'] - df_visits['outtime'])<=range_7d)


## 1. deathtime - hos discharge time
## 2. hospital mortality
## 3. note: include ED_death

In [16]:
df_visits.head()
##XF： more variables: 
## health utilization: n_ed_30d, n_ed_90d, n_ed_365. n_admission_*, n_icu_*, 
## keep chief_complaint, but added top-10 chief complaint as one-hot vector by keep word
## derive variable: next ED visit: ED revisit 72h
## icu outcome should be direct transfer to ICU or within 12hour after ED outtime. 
## inpatient mortality: check whether we use dod in patient.csv or deathtime in admission.csv
## race and insurance?? link to admission table?


## order: After finish the master dataset: keep all possibly useful information as much as possible, as orginal as possible. 
## further processing will be done in the next script: General_preprocessing


## user-friendly guidebook especially for beginer user. variable_name（triage_sbp, cci_*, chiefcom_*, ）, 


## ICD coding: ECI and CCI
## medical counts
## vital signs (last measurement for our demostration)


## extract_master_dataset.ipynb

## work on next script: general_processing.ipynb: 
# After each ipynb: provide number of ED visits/admissions/unique patients/critical care events/revisits/

#1. general filter: Exclude pediatric, or missing outcomes/a lot of features
#2. outliers
#3. train/test spliting
#4. missing values: imputation
#output: train.csv, test.csv?


## task specific dataset generation + modeling.ipynb

##1. at_triage_admission: exclude death within ED before disposition 
##2. at_triage_critical: 
##3. at_disposition_admission: exclude death within ED before disposition 
##4. at_disposition_critical: exclude death within ED before disposition 
##5. at_disposition_revisit: exclude death within ED before disposition/ exclude death within 72 hours before revisit/exclude patient admitted
## add variable: next time visit datetime -- then calculate 72h revisit(master_data.ipynb)


## time_series/ICD emebedding.ipynb


,index,subject_id,hadm_id,stay_id,intime,outtime,gender,age,deathtime,n_ed,...,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,admitted,icu,death_2day,death_7day
0,335314,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,52,NaT,0,...,97.0,106.0,63.0,0.0,3.0,"Abd pain, Abdominal distention",True,False,False,False
1,335315,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,52,NaT,1,...,97.0,116.0,88.0,10.0,3.0,Abdominal distention,True,False,False,False
2,335316,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,52,NaT,2,...,97.0,71.0,43.0,7.0,2.0,Hypotension,True,False,False,False
3,335317,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,52,NaT,3,...,98.0,96.0,50.0,13.0,2.0,"Abdominal distention, Abd pain, LETHAGIC",True,False,False,False
4,335318,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,52,NaT,4,...,96.0,106.0,57.0,10.0,3.0,"n/v/d, Abd pain",True,False,False,False


In [17]:
df_visits.to_csv('df_visits.csv', index=False)
